---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [3]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
len(spam_data)

5572

In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [5]:
def answer_one():
    spam_count = len(spam_data[spam_data['target']==1])
    total = len(spam_data)
    percentage = 100*(spam_count/total)
    return percentage

In [6]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    
       
    return sorted(vect.vocabulary_.keys(), key=len)[-1]

In [8]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_vector_train= vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1).fit(X_vector_train, y_train)
    predictions = clf.predict(vect.transform(X_test))
    auc_score = roc_auc_score(y_test, predictions)
    
    return auc_score

In [10]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [203]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    feature_names = np.array(vect.get_feature_names())
    
    tfidf_vals = X_train_vectorized.max(axis=0).toarray().ravel()
    dict1 = dict(zip(feature_names, tfidf_vals))
    
    df = pd.DataFrame([dict1.keys(), dict1.values()]).transpose().sort_values([1,0])
    
    smallest = pd.Series(df[1].iloc[:20].tolist(), index = df[0].iloc[:20])
    smallest.name = None
    smallest.index.name = None
    
    biggest = pd.Series(df[1].iloc[-21:-1].tolist(), index = df[0].iloc[-21:-1])
    biggest.name = None
    biggest.index.name = None
    
    return (smallest, biggest)

In [204]:
answer_four()

same         0.932467
blank        0.932702
tick         0.980166
146tf150p    1.000000
645          1.000000
anything     1.000000
anytime      1.000000
beerage      1.000000
done         1.000000
er           1.000000
havent       1.000000
home         1.000000
lei          1.000000
nite         1.000000
ok           1.000000
okie         1.000000
thank        1.000000
thanx        1.000000
too          1.000000
where        1.000000
dtype: float64

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [22]:
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vectorized = vect.transform(X_train)
    clf = MultinomialNB(alpha=0.1).fit(X_train_vectorized, y_train)
    predictions = clf.predict(vect.transform(X_test))
    auc_score = roc_auc_score(y_test, predictions)
    
    return auc_score

In [116]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [167]:
def answer_six():
    not_spam = spam_data[spam_data['target']==0]
    spam= spam_data[spam_data['target']==1]
    spam_length = spam['text'].str.len().mean()
    not_spam_length = not_spam['text'].str.len().mean()
    return (not_spam_length, spam_length)

In [168]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [129]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [178]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_vector_train = vect.transform(X_train)
    X_vector_trained = add_feature(X_vector_train, X_train.str.len())
    X_vector_test = add_feature(vect.transform(X_test), X_test.str.len())
    clf = SVC(C=10000).fit(X_vector_trained, y_train)
    predictions= clf.predict(X_vector_test)
    auc_score = roc_auc_score(y_test, predictions)
    
    return auc_score

In [179]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [188]:
def count_digits(string):
    return sum(s.isdigit() for s in string)

In [202]:
def answer_eight():
    not_spam = spam_data[spam_data['target']==0]
    spam = spam_data[spam_data['target']==1]
    spam_digits = spam['text'].apply(count_digits).mean()
    not_spam_digits = not_spam['text'].apply(count_digits).mean()
    return (not_spam_digits, spam_digits)

In [203]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [233]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_vector = vect.transform(X_train)
    X_trained_vector = add_feature(X_vector,X_train.str.len())
    X_test_vector = add_feature(vect.transform(X_test),X_test.str.len())
    
    X_trained_vector2 = add_feature(X_trained_vector, X_train.apply(lambda string: sum(s.isdigit() for s in string)))
    X_test_vector2 = add_feature(X_test_vector, X_test.apply(lambda string: sum(s.isdigit() for s in string)))   
    
    clf = LogisticRegression(C=100).fit(X_trained_vector2, y_train)
    predictions = clf.predict(X_test_vector2)
    auc_score = roc_auc_score(y_test, predictions)
    return auc_score

In [234]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [5]:
def answer_ten():
    import re
    spam= spam_data[spam_data['target']==1]
    not_spam = spam_data[spam_data['target']==0]
    
    spam_non_word = spam['text'].str.extractall(r'(\W)').count()[0]
    spam_total_docs = len(spam['text'])
    spam_non_word_avg = spam_non_word/spam_total_docs
    
    not_spam_non_word = not_spam['text'].str.extractall(r'(\W)').count()[0]
    not_spam_total_docs = len(not_spam['text'])
    not_spam_non_word_avg = not_spam_non_word/not_spam_total_docs
    
    return (not_spam_non_word_avg, spam_non_word_avg)

In [6]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [62]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
   
    #Convert X_train to a vectorized form
    X_vect = vect.transform(X_train)
    X_vect2 = add_feature(X_vect, X_train.str.len())
    X_vect3 = add_feature(X_vect2, X_train.apply(lambda string: sum(s.isdigit() for s in string)))
    X_vect4 = add_feature(X_vect3, X_train.str.count(r'(\W)'))
    
    #Convert X_test to a vectorized form
    Xtest_vect = vect.transform(X_test)
    Xtest_vect2 = add_feature(Xtest_vect, X_test.str.len())
    Xtest_vect3 = add_feature(Xtest_vect2, X_test.apply(lambda string: sum(s.isdigit() for s in string)))
    Xtest_vect4 = add_feature(Xtest_vect3, X_test.str.count(r'(\W)'))
    
    #Create a Logistic Regression model
    clf = LogisticRegression(C=100).fit(X_vect4, y_train)
    predictions = clf.predict(Xtest_vect4)
    auc_score = roc_auc_score(y_test, predictions)
    
    #Biggest and Smallest Coefficients
    feature_names = np.array(vect.get_feature_names())
    coef_index_sorted = clf.coef_[0].argsort()
    smallest = feature_names[coef_index_sorted[:10]].tolist()
    biggest = feature_names[coef_index_sorted[-11:-1]].tolist()
    
    return (auc_score, smallest, biggest)


In [63]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['eply ', 'ar', 'ww', ' x', 'mob', ' ch', 'xt', 'co', 'ia', 'ne'])